# Rule #1 Stock Screener
### Data Engineering Capstone Project

#### Project Summary
There are various ways to make investment decisions on the stockmarket. Many are based on data analysis.

One investment strategy which became wide-known due to well-known proponents such as Benjamin Graham and Warren Buffet is called [value investing] (https://en.wikipedia.org/wiki/Value_investing). In layman's terms it assumes that:
- via fundamental analysis an investor can buy stocks at less than their intrinsic value. 
- the intrinsic value will however in the long time be recognised by the market.

[Fundamental analysis] (https://en.wikipedia.org/wiki/Fundamental_analysis) is  done by calculating and evaluating KPIs from the financial statements of businesses. From these KPIs the intrinsic value of the stock is then derived. One guideline on how to do this is provided by [Phil Town in his book "Rule #1"] (https://en.wikipedia.org/wiki/Phil_Town).

He breaks the [relevant KPIs for value growth down to 5] (https://medium.datadriveninvestor.com/the-rule-1-of-long-term-investing-5e34c5702e49):
- Return on Investment Capital (ROIC)
- Sales growth rate
- Earnings per Share (EPS) growth rate
- Book Value per Share (BVPS) or Equity, growth rate
- Free Cash Flow (FCF or Cash) growth rate

Additionally Phil Town provides a formula on how to calculate the intrinisic value. This value he calles the "sticker price". An example calculation is available [here] (https://meetinvest.com/glossary/sticker-price#:~:text=where%20future%20market%20price%20%3D%20future,%2FE%20*%20estimated%20future%20EPS.).


This project aims to provide curated data assets on stocks traded in the NASDAQ exchange for an investment analyst in order to:
- inform on the industry background of a company and its peers.
- conduct fundamental analysis based on the rule #1 kpi set. 
- evaluate value growth KPIs and derive the sticker price.
- screen markets for stocks whose prices is under their intrinsic value.
- enable further optimization and backtesting via historic market price data.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simfin as sf

In [2]:
# Local spark cluster specific imports for Windows
import findspark
findspark.init()

In [4]:
# spark specific import. 
# Note: wait until run of previous cell is complete to avoid start-up issues.
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [5]:
# import of functions from custom modules. 
# Note: wait until run of previous cell is complete to avoid start-up issues.
from get_prices import *
from get_fundamentals import *
from get_peers import *
from get_company_info import *
from analysis_utils import *

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

After some initial research it was found that the required data on 
- company information
- financial statements 
- historical prices

is available via APIs of various providers. 

The financial statement and price data is then used to create the following additional data sets:
- growth KPIs: KPIs relevant for the growth #1 investment strategy
- screener results: identified stock which current prices is below the calculated intrinsic value.

The data engineering pipelines built in this project process the data in two steps:

1. Extraction from source and load to staging folder.
2. Extraction from staging folder, transformation and load to target folders.

The goal is to provide all tables as files in a target folder from which they will be loaded to an analytical tool for the evaluation.

Since all providers chosen for this project offer a python API the first pipeline for source data extraction is purely realized via python shell scripts.

However since the amount of data is quite largely (ca. 10 million records on historical stock pices), the second data pipeline which processes the data and creates the output files uses a combination of python and PySpark.

The project was entirely developed to run locally on a Windows computer. Consequently some specific imports and start-up procedures need to be followed in order to ensure a smooth run.

### Describe and Gather Data 
Below the various data sets and sources are described in the categories 
- company information
- financial statements 
- historical prices

Furthermore an overview is given on the definition of the content in the data sets:
- growth KPIs
- screener results

Describe the data sets you're using. Where did it come from? What type of information is included? 

#### 1.1 Company information
- Ticker symbol list: "http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt"
    - [Symbol Look-Up/Directory Data Fields & Definitions] (http://www.nasdaqtrader.com/trader.aspx?id=symboldirdefs)
    - "Ticker" symbol of a stock is the primary/ foreign key which connects all of the tables with each other.
    - it is filtered to only those Ticker symbols which
        - are not test issues
        - are stocks and not ETFs
        - are not bankrupt
- Company information: Kaggle dataset from 2019 https://www.kaggle.com/marketahead/all-us-stocks-tickers-company-info-logos 
    - this data was retrieved from the IEX Cloud API.
    - in order to save costs it was decided to re-use this dataset instead of downloading the data fresh from the API.
- Peer group information: IEX Cloud API https://iexcloud.io/docs/api/#peer-groups
    - the corresponding python API package is [pyEX] (https://pyex.readthedocs.io/en/latest/#)
    - this data is merged to the company information data set.

#### 1.2 Fundamental indicators 

- all fundamental data is retrieved via the [SimFin] API (https://simfin.com/contribute/overview#/)
    - the corresponding python API package is [simfin] (https://github.com/SimFin/simfin)
- the used financial statements are:
    - Income statement
    - Balance sheet
    - Cashflow statement

From these data sets the 5 relevant  KPIs listed by Phil Town are calculated:
- Return on Investment Capital (ROIC)
    - Source: Income Statement
        - net income: n_i -> column 'Net Income'
        - dividend: di -> column 'Dividends Paid'
        - debt: de -> summed up with equity in column 'Total Liabilities & Equity'
        - equity: eq
    - Definition: ROIC = (ni - di) / (de + eq)
- Sales Growth Rate
    - Source: Income Statement
    - Definition:  Sales is equal to column 'revenue'
- Earnings per Share Growth Rate
    - Source: Income Statement
        - Earnings: ea -> column 'Net Income'
        - number of shares : sh -> column 'Shares (Basic)'
    - Definition: ea / sh
- Book Value per Share Growth Rate
    - Source: Balance Sheet
        - Total Equity: t_e -> column 'Total Equity'
        - Prefered Equity: p_e (not available)
        - number of shares : sh -> column 'Shares (Basic)'
    - Definition: (t_e - p_e) / sh
- Free Cash Flow Growth Rate
    - Source: 
        - Cash Flow: 
            - Cashflow from Operating Activities: cf_oa -> column 'Net Cash from Operating Activities'
            - Capital Expenditure: capex -> column 'Net Cash from Investing Activities'
        - Income Statement:
            - Interest Expenses: i_e -> excluded for simplicity, column 'Interest Expense, Net'
            - Tax shield on Interest Expense: t_i_e  -> excluded for simplicity
    - Definition: Free Cashflow (f_cf) = cf_oa + i_e - t_i_e - capex


#### 1.3 Pricing information ####
The purpose of this data is to evaluate the so-called sticker prices and margin of safety based on current stock prices. It can also be used for backtesting criteria on historic data.
- all historic price data is retrieve via the Yahoo Finance API
    - the corresponding python package is [yfinance] (https://pypi.org/project/yfinance/)
    - for the data extraction an [existing script by Oleh Onyshchak] (https://www.kaggle.com/jacksoncrow/download-nasdaq-historical-data) was used and adapted.


#### 1.4 Growth KPI ####

From the fundamental KPIs growth indicators are derived which represent the performance over time. From these the intrinsic value of a stock is calculated via the "sticker price". An additional margin of safety is added to that.
- Sticker price calculation
    - future period fp, by default 10 years
    - Sticker price = future market price / (1 + exp_rr)^fp
    - expected annual return rate exp_rr, by default 15%
    - future market price = future P/E * estimated future EPS
        - future P/E = min(pe_default, pe_5yr_avg)
            - default price per earnings pe_default: 2* rule #1 growth rate (see below)
            - 5 year average of annual price per earnings pe_5yr_avg
                - annual price per earnings pe = price/ eps
                    - annual price = mean of daily low prices in month December
        - estimated future EPS f_eps = current EPS * (1+ rule1_gr)^fp
            - rule #1 growth rate rule1_gr = min(bvps_gr_5yr, eps_gr_5yr)
- Margin of safety: half the the sticker price.

#### 1.5 Screener results ####
This table is where things get interesting. The purpose of this table is to show all stocks which latest low price is below the calculated intrinsic value (including a margin of safety). For these stocks we get "value" for our bucks.

The table is generated by:
- extracting the latest stock price data for all stocks.
- joining the intrinsic values for each stock from the growth kpi table.
- filtering and keeping only those stocks which price is below the margin of safety value.


## Step 2: Explore and Assess the Data
#### Explore the Data ####
For the data exploration purposes the python package pandas profiling is used.
It generates a so called profiling report for a Pandas Dataframe.

Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### 2.1 Company information
#### 2.1.1 Ticker Symbol List

In [5]:
# get ticket symbol list
symbol_df = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
symbol_df.head(3)

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N
2,Y,AAA,Listed Funds Trust AAF First Priority CLO Bond...,P,,Y,100.0,N,NaN,AAA,AAA,N


In [6]:
print('Number of symbol in NASDAQ list before any filters: {}'\
    .format(symbol_df['NASDAQ Symbol'].nunique()))
# exclude test issues
symbol_df = symbol_df[(symbol_df['Test Issue'] == 'N')]
# exclude companies that are bankrupt
symbol_df = symbol_df[symbol_df['Financial Status'].isna() | (symbol_df['Financial Status']=='N')]
# exclude ETFs
symbol_df = symbol_df[symbol_df['ETF']=='N']
symbol_list = symbol_df['NASDAQ Symbol'].tolist()
print('Number of symbol in NASDAQ list after filters: {}'\
    .format(len(symbol_list)))

Number of symbol in NASDAQ list before any filters: 10657
Number of symbol in NASDAQ list after filters: 8129


#### 2.1.2 Company Info
From the Pandas Profiling report the following insights can be gathered:
- data for 4559 ticker symbols is available
- the amount of missing data is quite low (5.3%) and mostly on the columns logo, ceo and tag.
- the data quality is considered high, further cleaning is not required.

In [7]:
company_info_df = load_company_info_from_disk(symbol_list)
company_info_df.head(1)

Number of stocks symbols in list: 4545
Company data loaded from disk...


,ticker,company name,short name,industry,description,website,logo,ceo,exchange,market cap,sector,tag 1,tag 2,tag 3
0,A,Agilent Technologies Inc.,Agilent,Medical Diagnostics & Research,Agilent Technologies Inc is engaged in life sc...,http://www.agilent.com,A.png,Michael R. McMullen,New York Stock Exchange,2.421807e+10,Healthcare,Healthcare,Diagnostics & Research,Medical Diagnostics & Research


In [ ]:
create_pandas_profiling_report(company_info_df, 'company_info_df')

In [8]:
# reduce symbol_list to those where company information is available
symbol_list = company_info_df['ticker'].unique().tolist()

#### 2.1.3 Peer group information
For retrieving this data via the API there is a cost per ticker symbol. In order to limit these costs it was decided to limit data retrieval to only those symbols which are in the company information data set (4545 ticker symbols in total).

From Pandas Profiling report the following insights can be gathered:
- an API call was successfull for 4536 out of the 4545 ticker symbols.
- For 3375 of these ticker symbols the peer data is available.

Since the peer group is considered a supplementary information, it was decided to proceed in the project with the total number of symbols in the company information data set. The amount of missing values for peer group is acceptable.

In [17]:
# initial download of peer data from API
# Note: requires a valid API key
#peer_df = download_peer_data(symbol_list)

In [9]:
# load downloaded peer data from disk
peer_df = get_peer_data_from_disk(symbol_list)
peer_df_shape = peer_df.shape
print(f'Shape of peer_df: {peer_df_shape}')

Shape of peer_df: (4536, 3)


In [10]:
create_pandas_profiling_report(peer_df, 'peer_df')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 143.00it/s]
Pandas profiling report of file peer_df created





In [14]:
# add peer data to company info
company_info_df = company_info_df.merge(peer_df, 
                                        on = 'ticker',
                                        how='left',
                                        validate='1:1')
company_info_df.head(3)

,ticker,company name,short name,industry,description,website,logo,ceo,exchange,market cap,sector,tag 1,tag 2,tag 3,peer_string,peer_list
0,A,Agilent Technologies Inc.,Agilent,Medical Diagnostics & Research,Agilent Technologies Inc is engaged in life sc...,http://www.agilent.com,A.png,Michael R. McMullen,New York Stock Exchange,2.421807e+10,Healthcare,Healthcare,Diagnostics & Research,Medical Diagnostics & Research,"TMO,PKI,DHR,TER,NATI,ILMN,AME,BRKR,GE,SPMYY","[TMO, PKI, DHR, TER, NATI, ILMN, AME, BRKR, GE..."
1,AA,Alcoa Corporation,Alcoa,Metals & Mining,Alcoa Corp is an integrated aluminum company. ...,http://www.alcoa.com,AA.png,Roy Christopher Harvey,New York Stock Exchange,5.374967e+09,Basic Materials,Basic Materials,Aluminum,Metals & Mining,"ACH,KALU,CENX,NHYDY,AWCMY,BBL,BHP","[ACH, KALU, CENX, NHYDY, AWCMY, BBL, BHP]"
2,AAC,AAC Holdings Inc.,AAC,Health Care Providers,AAC Holdings Inc provides inpatient and outpat...,http://www.americanaddictioncenters.org,NaN,Michael T. Cartwright,New York Stock Exchange,6.372010e+07,Healthcare,Healthcare,Medical Care,Health Care Providers,"SEM,ACHC,USPH,BICX","[SEM, ACHC, USPH, BICX]"


### 2.2 Fundamental indicators
The Simfin API others a bulk download of fundamental data traded on American stock exchanges.
From the Pandas Profiling report the following insights can be gathered:
- Income statement: 
    - data for 2296 ticker symbols available.
    - column "Revenue" has low number of missing values.
    - Fiscal Years: data availability after 2010 is good, but no before.
- Balance sheet:
    - data for 2297 ticker symbols available.
    - Fiscal Years: data availability after 2010 is good, but no before.
- Cashflow statement:
    - data for 2296 ticker symbols available.
    - Fiscal Years: data availability after 2010 is good, but no before.
    - column "Shares (Basic)" has low number of missing values.
    - column "Dividents Paid" has high number of missing values (50%).
        - data cleansing: NA values are replaced with Zero.
        - this way the missing data has no effect on the overall ROIC calculation.
        - this is however a cause of potential errors in the KPI.
    - column "Net Cash from Investing Activities" has low number of missing values.

However the number of Ticker symbols with data overlapping with those in the symbol list derived from the company information data set is considerably lower:
- Available symbols in data set cashflow_df: 1660 of total 4545
- Available symbols in data set income_sm_df: 1660 of total 4545
- Available symbols in data set balance_st_df: 1661 of total 4545

Since the fundamental data is at the core of value investing strategies, the symbol list will be reduced to the 1660 tickers with available fundamental data

In [9]:
# initialize simfin API
init_simfin_api()

In [10]:
market='us'
variant='annual'
# download cashflow data from the SimFin server and load into a Pandas DataFrame.
cashflow_df = sf.load_cashflow(variant=variant, market=market)
cashflow_df = cashflow_df.reset_index()
# Download the data from the SimFin server and load into a Pandas DataFrame.
income_sm_df = sf.load_income(variant=variant, market=market)
income_sm_df = income_sm_df.reset_index()
# Download the data from the SimFin server and load into a Pandas DataFrame.
balance_st_df = sf.load_balance(variant=variant, market=market)
balance_st_df = balance_st_df.reset_index()

Dataset "us-cashflow-annual" on disk (28 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (28 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (25 days old).
- Loading from disk ... Done!


In [ ]:
# create Pandas Profiling Report for each DataFrame
for df, df_name in zip([cashflow_df, income_sm_df, balance_st_df],
                        ['cashflow_df', 'income_sm_df', 'balance_st_df']):
    create_pandas_profiling_report(df, df_name)

In [11]:
# check availability of fundamental data for stocks in symbol list
for df, df_name in zip([cashflow_df, income_sm_df, balance_st_df],
                        ['cashflow_df', 'income_sm_df', 'balance_st_df']):
    symbol_cnt = len(symbol_list)
    available_symbol_cnt = df[df['Ticker'].isin(symbol_list)]['Ticker'].nunique()
    print(f'Available symbols in data set {df_name}: {available_symbol_cnt} of total {symbol_cnt}')

Available symbols in data set cashflow_df: 1660 of total 4545
Available symbols in data set income_sm_df: 1660 of total 4545
Available symbols in data set balance_st_df: 1661 of total 4545


In [18]:
# determine time period for analysis
period_dict = {'start_date':2010,
                'end_date':2019}

In [21]:
# combine fundamentals and calculate top5 kpis
fundamental_df = combine_fundamentals(symbol_list)
# filter on relevant time period
fundamental_df = filter_df(fundamental_df, period_dict)
fundamental_df = calculate_top5_kpi(fundamental_df)
# reduce symbol_list to those where fundamental data is available
symbol_list = fundamental_df['Ticker'].unique().tolist()
fundamental_df.tail(3)

Dataset "us-cashflow-annual" on disk (28 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (28 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (26 days old).
- Loading from disk ... Done!
Symbols with available fundamental data: 1655
Combined all fundamental data from financial statements to one Dataframe.
Calculated roic and added it to Dataframe
Calculated eps and added it to Dataframe
Calculated bvps and added it to Dataframe
Calculated fcf and added it to Dataframe
top5 KPIs added to fundamental data


,Ticker,Report Date_is,SimFinId,Currency,Fiscal Year,Fiscal Period_is,Publish Date_is,Restated Date_is,Shares (Basic)_is,Shares (Diluted)_is,...,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Dividends Paid_clean,roic,eps,bvps,fcf
14387,ZYNE,2017-12-31,901704,USD,2017,FY,2018-03-12,2019-03-11,12914814.0,12914814.0,...,138930454.0,NaN,-77980866.0,60949588.0,69054309,0.0,-0.463582,-2.478727,4.719355,-25727095.0
14388,ZYNE,2018-12-31,901704,USD,2018,FY,2019-03-11,2019-03-11,15308886.0,15308886.0,...,175493702.0,NaN,-117892041.0,57601661.0,67327443,0.0,-0.592792,-2.607059,3.762629,-32110693.0
14389,ZYNE,2019-12-31,901704,USD,2019,FY,2020-03-10,2020-03-10,22000203.0,22000203.0,...,226432367.0,NaN,-150835624.0,75596743.0,87764596,0.0,-0.375363,-1.497422,3.436184,-34688586.0


### 2.3 Pricing information
Since the amount of available data is quite large, it is not possible to use Pandas Profiling here for an evaluation. Instead descriptive statistics are derived from PySpark computations
- Number of rows in entire price data:  9.702.060
- Data for 1655 out of 1660 ticker symbols is available via the API.
- the price data is essential for the growth KPI calculation. Consequently the processed data will again be restricted to the ticker symbols with available price data.

In [17]:
%%time
# Download historic stock prices for symbols
# Note: this might take a long time, consequently the code line below is commented.
#download_ticker_prices(symbol_list)

Wall time: 0 ns


In [7]:
%%time
# load ticker prices for symbols
price_df = load_ticker_prices(spark, symbol_list)
# print Schema
price_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Ticker: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: double (nullable = true)

Wall time: 16.1 s


In [8]:
# print tail and visualize in Pandas
price_df.limit(3).toPandas()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,HPQ,0.131273,0.131273,0.124177,0.124177,0.046594,2480333.0
1,1962-01-03,HPQ,0.124177,0.124177,0.121516,0.122846,0.046094,507341.0
2,1962-01-04,HPQ,0.122846,0.126838,0.117968,0.120185,0.045096,845568.0


In [9]:
%%time
# count number of rows
price_df_row_count = price_df.count()
print(f'Number of rows in price data: {price_df_row_count}')

Number of rows in price data: 9702060
Wall time: 13.3 s


In [10]:
%%time
price_df.select(F.countDistinct('Ticker')).show()

+----------------------+
|count(DISTINCT Ticker)|
+----------------------+
|                  1655|
+----------------------+

Wall time: 37.8 s


In [11]:
%%time
price_df.select(F.min('Date'),F.max('Date')).show()

+----------+----------+
| min(Date)| max(Date)|
+----------+----------+
|1962-01-02|2021-05-13|
+----------+----------+

Wall time: 21.2 s


In [22]:
%%time
# restrict symbol_list to those with available price data
symbol_list = price_df.select('Ticker').distinct().toPandas()['Ticker'].tolist()
# filter on relevant time period
fundamental_df = filter_df(fundamental_df, period_dict)

Wall time: 24.2 s


### 2.4 Growth KPIs
The growth KPI calculation is calculated from indicators in the fundamental and price data sets. Two more KPIs need to be added to the fundamental data set as prequisite:
- annual price data. In this project: mean low price of all trading days in month December.
- annual price per earning ratio.

Once this is done, alle growth KPIs that are necessary for a calculation of the intrinsic stock value can be calculated from the fundamental data set.

In [19]:
# calculate annual price from historic price data
ann_price_df = calculate_annual_price(spark, price_df, period_dict)
ann_price_df.tail(3)

,year,Ticker,mean_low_price
14485,2019,ZUMZ,31.880000
14486,2019,ZUO,14.341429
14487,2019,ZYNE,5.738524


In [23]:
# calculate annual price per earnings
fundamental_df = calculate_annual_pe(ann_price_df, fundamental_df)
fundamental_df.tail(3)

,Ticker,Report Date_is,SimFinId,Currency,Fiscal Year,Fiscal Period_is,Publish Date_is,Restated Date_is,Shares (Basic)_is,Shares (Diluted)_is,...,Retained Earnings,Total Equity,Total Liabilities & Equity,Dividends Paid_clean,roic,eps,bvps,fcf,mean_low_price,pe
14387,ZYNE,2017-12-31,901704,USD,2017,FY,2018-03-12,2019-03-11,12914814.0,12914814.0,...,-77980866.0,60949588.0,69054309,0.0,-0.463582,-2.478727,4.719355,-25727095.0,11.937900,-4.816141
14388,ZYNE,2018-12-31,901704,USD,2018,FY,2019-03-11,2019-03-11,15308886.0,15308886.0,...,-117892041.0,57601661.0,67327443,0.0,-0.592792,-2.607059,3.762629,-32110693.0,3.816737,-1.464001
14389,ZYNE,2019-12-31,901704,USD,2019,FY,2020-03-10,2020-03-10,22000203.0,22000203.0,...,-150835624.0,75596743.0,87764596,0.0,-0.375363,-1.497422,3.436184,-34688586.0,5.738524,-3.832270


In [25]:
# calculate growth kpi df
growth_df = calculate_growth_rates(fundamental_df, agg_func='mean')
growth_df.head(3)

Calculated KPI growth from year to year.
Calculated 5 and 10 year growth rate


,Ticker,revenue_gr_curr,eps_curr,roic_gr_5yr,revenue_gr_5yr,eps_gr_5yr,bvps_gr_5yr,fcf_gr_5yr,pe_5yr,yrs_in_5yr,...,pe_default_5yr,roic_gr_10yr,revenue_gr_10yr,eps_gr_10yr,bvps_gr_10yr,fcf_gr_10yr,pe_10yr,yrs_in_10yr,rule1_gr_10yr,pe_default_10yr
0,A,0.050672,3.410828,1.03,0.05,0.47,-0.00,0.24,38.06,5.0,...,NaN,0.54,0.01,0.26,0.07,-11.94,26.45,10,0.07,14.0
1,AA,-0.221592,-6.081081,-2.08,0.00,-2.01,-0.14,3.41,7.76,5.0,...,NaN,-2.08,0.00,-2.01,-0.14,3.41,7.76,5,NaN,NaN
2,AAL,0.027548,3.802753,0.05,0.01,0.22,-0.09,0.09,8.91,5.0,...,NaN,0.08,0.10,1.00,0.10,0.15,4.37,10,0.10,20.0


In [27]:
growth_df = calculate_sticker_price(growth_df, fp=10, exp_rr=0.15)
growth_df.head(3)

,Ticker,revenue_gr_curr,eps_curr,roic_gr_5yr,revenue_gr_5yr,eps_gr_5yr,bvps_gr_5yr,fcf_gr_5yr,pe_5yr,yrs_in_5yr,...,fcf_gr_10yr,pe_10yr,yrs_in_10yr,rule1_gr_10yr,pe_default_10yr,pe_future,eps_future,price_future,sticker_price,mos
0,A,0.050672,3.410828,1.03,0.05,0.47,-0.00,0.24,38.06,5.0,...,-11.94,26.45,10,0.07,14.0,38.06,NaN,NaN,NaN,NaN
1,AA,-0.221592,-6.081081,-2.08,0.00,-2.01,-0.14,3.41,7.76,5.0,...,3.41,7.76,5,NaN,NaN,7.76,NaN,NaN,NaN,NaN
2,AAL,0.027548,3.802753,0.05,0.01,0.22,-0.09,0.09,8.91,5.0,...,0.15,4.37,10,0.10,20.0,8.91,NaN,NaN,NaN,NaN


### 2.4 Screener results
The processing of the screener results table is comparatively easy:
- the latest prices are extracted from the price data set
- the relevant columns for the intrinsic value from the growth kpi data set are joined.
- tbd

In [29]:
%%time
# find stocks which latest price is under the mos
screener_df = find_stocks_below_mos(spark, price_df, growth_df)

Wall time: 3min 24s


## Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
- Dimensional tables
    - company information, including peers
- Fact tables:
    - fundamental indicators
    - price history
    - growth KPIs
    - screener results

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
- Data extraction pipeline: extract data from sources via APIs and store results in staging tables.
    - NASDAAQ: stock symbol list
    - IEX Cloud source:
        - Company information data
        - Peer group data
    - Simfin source: fundamental data
        - Annual cashflow data
        - Annual income statement data
        - Annual balance sheet data
    - Yfinance source: historical stock price data
- Data processing pipeline: process data from staging tables to dimension and fact tables.
    - create company information dimension table with symbol list.
    - create price facts table
    - create fundamental facts table
    - create growth KPI facts table
    - create screener results KPI table

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [6]:
%%time
# download data from sources to staging folders
symbol_list = pipeline_staging()
print('Number of symbols in list after staging pipeline ran: {}'.format(len(symbol_list)))

Symbol data extracted...
total number of symbols traded = 8129
Number of stocks symbols in list: 4545
Company data loaded from disk...
Dataset "us-cashflow-annual" on disk (28 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (28 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (26 days old).
- Loading from disk ... Done!
Symbols with available fundamental data: 1660
Combined all fundamental data from financial statements to one Dataframe.
Start download of historic price data
Ticker price data extracted...
Total number of valid symbols downloaded = 1655
Staging pipeline run complete.
Number of symbols in list after staging pipeline ran: 1660
Wall time: 28min 28s


In [7]:
%%time
period_dict = {'start_date':2010,
                'end_date':2019}

price_df, company_info_df, fundamental_df, growth_df, screener_df = pipeline_processing(spark, period_dict)
growth_df[growth_df['Ticker']=='CARS'].iloc[0]

Number of stocks symbols in list: 6368
Company data loaded from disk...
Dataset "us-cashflow-annual" on disk (23 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (23 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (20 days old).
- Loading from disk ... Done!
Combined all fundamental data from financial statements to one Dataframe.
Calculated roic and added it to Dataframe
Calculated eps and added it to Dataframe
Calculated bvps and added it to Dataframe
Calculated fcf and added it to Dataframe
top5 KPIs added to fundamental data
Calculated KPI growth from year to year.
Calculated 5 and 10 year growth rate


Ticker                  CARS
revenue_gr_curr    -0.083738
eps_curr           -6.647123
roic_gr_5yr        -5.420000
revenue_gr_5yr     -0.010000
eps_gr_5yr         -4.530000
bvps_gr_5yr              NaN
fcf_gr_5yr         -0.140000
pe_5yr             15.900000
yrs_in_5yr          4.000000
rule1_gr_5yr             NaN
pe_default_5yr           NaN
roic_gr_10yr       -5.420000
revenue_gr_10yr    -0.010000
eps_gr_10yr        -4.530000
bvps_gr_10yr             NaN
fcf_gr_10yr        -0.140000
pe_10yr            15.900000
yrs_in_10yr                4
rule1_gr_10yr            NaN
pe_default_10yr          NaN
pe_future          15.900000
eps_future               NaN
price_future             NaN
sticker_price            NaN
mos                      NaN
Name: 298, dtype: object

In [9]:
screener_df.head()

,Ticker,last_date,last_low_price,price_future,sticker_price,mos
0,ALXN,2021-05-05,168.789993,11071.685791,2736.751398,1368.375699
1,ABMD,2021-05-05,299.779999,2648.558973,654.683271,327.341636
2,SABR,2021-05-05,12.410000,11930.046852,2948.925125,1474.462563
3,FND,2021-05-05,111.550003,2714.119248,670.888769,335.444384
4,EA,2021-05-05,137.630005,1500.221008,370.831689,185.415845


In [12]:
eval_df = screener_df.merge(company_info_df.rename(columns={'ticker':'Ticker'}),
                        how='left',
                        on='Ticker',
                        validate='1:1')
eval_df.head(2)

,Ticker,last_date,last_low_price,price_future,sticker_price,mos,company name,short name,industry,description,...,logo,ceo,exchange,market cap,sector,tag 1,tag 2,tag 3,peer_string,peer_list
0,ALXN,2021-05-05,168.789993,11071.685791,2736.751398,1368.375699,Alexion Pharmaceuticals Inc.,Alexion Pharmaceuticals,Biotechnology,Alexion Pharmaceuticals Inc is a biopharmaceut...,...,ALXN.png,Ludwig N. Hantson,Nasdaq Global Select,2.760821e+10,Healthcare,Healthcare,Biotechnology,NaN,"GILD,REGN,VRTX,BIIB,QGEN,AGIO,RARE,SRPT,BIO.B,JNJ","[GILD, REGN, VRTX, BIIB, QGEN, AGIO, RARE, SRP..."
1,ABMD,2021-05-05,299.779999,2648.558973,654.683271,327.341636,ABIOMED Inc.,ABIOMED,Medical Devices,Abiomed Inc is a medical device company. It pr...,...,ABMD.png,Michael R. Minogue,Nasdaq Global Select,1.488763e+10,Healthcare,Healthcare,Medical Devices,NaN,"TFX,BSX,STXS,ATRC","[TFX, BSX, STXS, ATRC]"


In [25]:
eval_df.to_excel('../data/4_data_analysis/' + str(pd.to_datetime('today'))[:10] + '_eval_df.xlsx', index=False)

In [27]:
eval_df['mos'].describe(percentiles=[.25, .5, .75, .8, .9, .95])

count    8.900000e+01
mean     1.932307e+23
std      1.822935e+24
min      4.376009e+00
25%      9.204744e+01
50%      3.273416e+02
75%      1.474463e+03
80%      2.305800e+03
90%      8.875529e+03
95%      7.727330e+06
max      1.719753e+25
Name: mos, dtype: float64

In [28]:
growth_df[growth_df['Ticker']=='FIVN'].iloc[0]

Ticker                    FIVN
revenue_gr_curr       0.272999
eps_curr             -0.075400
roic_gr_5yr           2.740000
revenue_gr_5yr        0.260000
eps_gr_5yr            3.300000
bvps_gr_5yr           0.450000
fcf_gr_5yr            5.060000
pe_5yr            -2357.840000
yrs_in_5yr            5.000000
rule1_gr_5yr          0.450000
pe_default_5yr       90.000000
roic_gr_10yr          2.220000
revenue_gr_10yr       0.250000
eps_gr_10yr           2.610000
bvps_gr_10yr         -0.040000
fcf_gr_10yr           4.080000
pe_10yr           -1965.590000
yrs_in_10yr                  7
rule1_gr_10yr              NaN
pe_default_10yr            NaN
pe_future         -2357.840000
eps_future           -3.097804
price_future       7304.125676
sticker_price      1805.468159
mos                 902.734079
Name: 641, dtype: object

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [28]:
# Perform quality checks here

In [25]:
# filter for verification of mean calculation
# price_df.filter(
#                     (price_df['Date']>=F.to_date(F.lit('2020-12-01'))) &\
#                     (price_df['Date']<=F.to_date(F.lit('2020-12-31'))) &\
#                     (price_df['Ticker']=='AAPL')
#                 ).toPandas()['Low'].mean()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [12]:
# TODO: create data dictionary

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [6]:
# export notebook to markdown for documentation
!jupyter nbconvert --to markdown capstone_stockscreener.ipynb

[NbConvertApp] Converting notebook capstone_stockscreener.ipynb to markdown
[NbConvertApp] Writing 58833 bytes to capstone_stockscreener.md
